<h3>Importing the needed libraries</h3>

In [2]:
import pandas as pd, seaborn as sns, matplotlib.pyplot as plt,numpy as np
from sklearn.preprocessing import StandardScaler, MinMaxScaler, MaxAbsScaler,QuantileTransformer
from rapidfuzz import process,fuzz
import pandas as pd
import os
import numpy as np

<h3>Defining the movie querier function</h3>

In [3]:
import requests
import re
from rapidfuzz import fuzz, process
import numpy as np

def get_name_movie(queried_movie_name, train_movies_1d, movie_type):
    query_method = None
    
    #This function compiles the queried movie data into the format needed(TV-SHOW ONLY)
    def compile_meta_show(imdbID=np.nan, title=np.nan, url=np.nan, imdbRating=np.nan, imdbVotes=np.nan, duration=np.nan,
                          year=np.nan, noOfAwards=np.nan, noOfNominations=np.nan, Action=0, Adult=0, Adventure=0,
                          Animation=0, Biography=0, Comedy=0, Crime=0, Documentary=0, Drama=0, Family=0, Fantasy=0,
                          FilmNoir=0, GameShow=0, History=0, Horror=0, Music=0, Musical=0, Mystery=0, News=0,
                          RealityTV=0, Romance=0, SciFi=0, Short=0, Sport=0, TalkShow=0, Thriller=0, War=0, Western=0,
                          directors=np.nan, writers=np.nan, actors=np.nan, country=np.nan, language=np.nan, Type=np.nan,
                          Plot=np.nan, Rated=np.nan, Genre=np.nan, totalSeasons=np.nan, specialNominations=np.nan):
        url = f'http://www.imdb.com/title/{imdbID}/'
        return [imdbID, title, url, imdbRating, imdbVotes, duration, year, specialNominations, noOfAwards,
                noOfNominations, Action, Adult, Adventure, Animation, Biography, Comedy, Crime, Documentary, Drama,
                Family, Fantasy, FilmNoir, GameShow, History, Horror, Music, Musical, Mystery, News, RealityTV, Romance,
                SciFi, Short, Sport, TalkShow, Thriller, War, Western, directors, writers, actors, country, language,
                Type, Plot, Rated, Genre, totalSeasons]
    
    #This function compiles the queried movie data into the format needed(MOVIES ONLY)
    def compile_meta(imdbID=np.nan, title=np.nan, url=np.nan, imdbRating=np.nan,
                     rottenTomatoRating=np.nan, metacriticRating=np.nan,
                     imdbVotes=np.nan, duration=np.nan, year=np.nan,
                     oscarNominations=np.nan, noOfAwards=np.nan, noOfNominations=np.nan, Action=0, Adult=0, Adventure=0,
                     Animation=0, Biography=0, Comedy=0, Crime=0, Documentary=0, Drama=0, Family=0, Fantasy=0,
                     FilmNoir=0, GameShow=0, History=0, Horror=0, Music=0, Musical=0, Mystery=0, News=0,
                     RealityTV=0, Romance=0, SciFi=0, Short=0, Sport=0, TalkShow=0, Thriller=0, War=0,
                     Western=0, directors=np.nan, writers=np.nan, actors=np.nan, production=np.nan,
                     country=np.nan, language=np.nan, Type=np.nan, Plot=np.nan, BoxOffice=np.nan, Rated=np.nan,
                     Genre=np.nan):
        url = f'http://www.imdb.com/title/{imdbID}/'
        return [imdbID, title, url, imdbRating, rottenTomatoRating, metacriticRating,
                imdbVotes, duration, year, oscarNominations, noOfAwards, noOfNominations,
                Action, Adult, Adventure, Animation, Biography, Comedy, Crime, Documentary,
                Drama, Family, Fantasy, FilmNoir, GameShow, History, Horror, Music, Musical,
                Mystery, News, RealityTV, Romance, SciFi, Short, Sport, TalkShow, Thriller,
                War, Western, directors, writers, actors, production, country, language,
                Type, Plot, BoxOffice, Rated, Genre
                ]
    #This function gets the meta information of the queried movie by using its IMDB ID
    def get_meta(movie_id, type_):
        url = "https://movie-database-imdb-alternative.p.rapidapi.com/"
        headers = {
            'x-rapidapi-host': "movie-database-imdb-alternative.p.rapidapi.com",
            'x-rapidapi-key': "ce15ebaf8dmshfb3801d1cfa22dcp1f4c05jsn1dad2b361974"
        }
        querystring = {"i": f"{movie_id}", "r": "json", 'type': type_}
        response = requests.request("GET", url, headers=headers, params=querystring)

        return (response.json())

    #This function gets the IMDB ID of a queried movie name
    def get_id(m_name, type_):
        url = "https://movie-database-imdb-alternative.p.rapidapi.com/"
        headers = {
            'x-rapidapi-host': "movie-database-imdb-alternative.p.rapidapi.com",
            'x-rapidapi-key': "ce15ebaf8dmshfb3801d1cfa22dcp1f4c05jsn1dad2b361974"
        }
        querystring = {"page": "1", "r": "json", "s": f"{m_name}", 'type': type_}
        response = requests.request("GET", url, headers=headers, params=querystring)

        return (response.json()['Search'][0]['imdbID'])

    try:
        m_id = get_id(queried_movie_name, movie_type)
        query_method = 'API Search'
    except KeyError as err:
        m_id = \
            process.extractOne(queried_movie_name.lower(), train_movies_1d, scorer=fuzz.token_set_ratio)[0].split(":")[
                -1]
        query_method = 'DATA Search'
    except ConnectionError as err:
        return (None, 'Connection Error')
    except Exception as err:
        return (None, err)
    try:
        movie_meta = get_meta(m_id, movie_type)
        if movie_type != "series":
            log_obj = dict()
            log_obj['imdbID'] = movie_meta['imdbID']
            log_obj['title'] = movie_meta['Title'].replace(',', "-")
            dict_ratings = {
                'Internet Movie Database': {'name': 'imdbRating', 'pattern': r'(.+?)/'},
                'Rotten Tomatoes': {'name': 'rottenTomatoRating', 'pattern': r'(.+?)%'},
                'Metacritic': {'name': 'metacriticRating', 'pattern': r'(.+?)/'}
            }
            for i, j in [(i['Source'], i['Value']) for i in movie_meta['Ratings']]:
                log_obj[dict_ratings[i]['name']] = float(re.findall(dict_ratings[i]['pattern'], j)[0])
            try:
                log_obj['imdbVotes'] = int(movie_meta['imdbVotes'].replace(',', ''))
            except:
                pass
            try:
                log_obj['duration'] = int(re.findall(r'(\d+)', movie_meta['Runtime'])[0]) * 60
            except:
                pass
            log_obj['year'] = movie_meta['Year']
            nomination_text = movie_meta['Awards']
            if 'Oscar' in nomination_text:
                log_obj['oscarNominations'] = int(re.findall(r'(\d+) Oscar', nomination_text, re.IGNORECASE)[0])
            if 'nominations' in nomination_text:
                log_obj['noOfNominations'] = int(
                    re.findall(r'(\d+) nominations', nomination_text, re.IGNORECASE)[0])
            if 'wins' in nomination_text:
                log_obj['noOfAwards'] = int(re.findall(r'(\d+) wins', nomination_text, re.IGNORECASE)[0])
            for i in movie_meta['Genre'].split(','):
                log_obj[i.replace(' ', '').replace('-', '')] = 1
            log_obj['directors'] = movie_meta['Director'].replace(",", '-').replace("N/A", '')
            log_obj['writers'] = movie_meta['Writer'].replace(",", '-').replace("N/A", '')
            log_obj['actors'] = movie_meta['Actors'].replace(",", '-').replace("N/A", '')
            try:
                log_obj['production'] = movie_meta['Production'].replace(",", '-').replace("N/A", '')
            except:
                pass
            log_obj['country'] = movie_meta['Country'].replace(",", '-').replace("N/A", '')
            log_obj['language'] = movie_meta['Language'].replace(",", '-').replace("N/A", '')
            log_obj['Plot'] = movie_meta['Plot'].replace(",", '-').replace("N/A", '')
            log_obj['Type'] = movie_meta['Type'].replace(",", '-').replace("N/A", '')
            log_obj['Rated'] = movie_meta['Rated'].replace(",", '-').replace("N/A", '')
            try:
                log_obj['BoxOffice'] = movie_meta['BoxOffice'].replace(",", '-').replace("N/A", '')
            except:
                pass
            log_obj['Genre'] = movie_meta['Genre'].replace(",", '-').replace("N/A", '')
            if 'N/A' in log_obj.keys():
                del log_obj['N/A']
            return (compile_meta(**log_obj), query_method, None)
        else:
            log_obj = dict()
            log_obj['imdbID'] = movie_meta['imdbID']
            log_obj['title'] = movie_meta['Title'].replace(',', "-")
            dict_ratings = {
                'Internet Movie Database': {'name': 'imdbRating', 'pattern': r'(.+?)/'}
            }
            for i, j in [(i['Source'], i['Value']) for i in movie_meta['Ratings']]:
                try:
                    log_obj[dict_ratings[i]['name']] = float(re.findall(dict_ratings[i]['pattern'], j)[0])
                except:
                    pass
            try:
                log_obj['imdbVotes'] = int(movie_meta['imdbVotes'].replace(',', ''))
            except:
                pass
            try:
                log_obj['duration'] = int(re.findall(r'(\d+)', movie_meta['Runtime'])[0]) * 60
            except:
                pass
            log_obj['year'] = movie_meta['Year']
            nomination_text = movie_meta['Awards']
            log_obj['specialNominations'] = 'None'
            log_obj['noOfNominations'] = 0
            log_obj['noOfAwards'] = 0
            if 'nominated for' in nomination_text.lower():
                for (value, nomination) in re.findall(r'(\d+?)\s([a-zA-Z\s]+)', nomination_text, re.IGNORECASE):
                    if not ('nomination' in nomination or 'win' in nomination):
                        log_obj['specialNominations'] += f'{value}:{nomination};'
            if 'nomination' in nomination_text:
                log_obj['noOfNominations'] = int(re.findall(r'(\d+) nomination', nomination_text, re.IGNORECASE)[0])
            if 'win' in nomination_text:
                log_obj['noOfAwards'] = int(re.findall(r'(\d+) win', nomination_text, re.IGNORECASE)[0])
            for i in movie_meta['Genre'].split(','):
                log_obj[i.replace(' ', '').replace('-', '')] = 1
            log_obj['directors'] = movie_meta['Director'].replace(",", '-')
            log_obj['writers'] = movie_meta['Writer'].replace(",", '-')
            log_obj['actors'] = movie_meta['Actors'].replace(",", '-')
            log_obj['country'] = movie_meta['Country'].replace(",", '-')
            log_obj['language'] = movie_meta['Language'].replace(",", '-')
            log_obj['Plot'] = movie_meta['Plot'].replace(",", '-')
            log_obj['Type'] = movie_meta['Type'].replace(",", '-')
            log_obj['Rated'] = movie_meta['Rated'].replace(",", '-')
            log_obj['Genre'] = movie_meta['Genre'].replace(",", '-')
            try:
                log_obj['totalSeasons'] = int(movie_meta['totalSeasons'])
            except:
                pass
            if 'N/A' in log_obj.keys():
                del log_obj['N/A']
            return (compile_meta_show(**log_obj), query_method, None)
    except ConnectionError as err:
        return (None, 'Connection Error')
    except Exception as err:
        raise err
        return (None, 'Unknown error encountered')


<h3>Defining the predicting algorithm</h3>

In [4]:
from rapidfuzz import fuzz,process
import math,re,numpy as np,pandas as pd
import re


def euclidean_distance(one_d_arr_1,one_d_arr_2):
	if (len(one_d_arr_2) != len(one_d_arr_2)):
		raise Exception('Eucidean Distance Error; the two 1D arrays have mismatched lengths')
	return np.linalg.norm(one_d_arr_2-one_d_arr_1)


def extract_values(one_d_arr, indices):
	return one_d_arr[indices]

def sort_template_by_rank(distance_template):
	return sorted(distance_template, key=lambda r:r[0])

def compute_distances(query_data_1d, train_data_2d, distance_template):
	sorted_distance_template = sort_template_by_rank(distance_template)
	distances_computed = list()
	pattern_d_ratio = re.compile('d(\d+)')
	for id_, data in enumerate(train_data_2d):
		distance_values =list()
		for *_, indices, method, d_ratio,multiplier in sorted_distance_template:
			values_query = extract_values(query_data_1d, indices)
			values_train = extract_values(data, indices)
			distance = euclidean_distance(values_query.astype(np.float64),values_train.astype(np.float64))
			if pattern_d_ratio.findall(d_ratio)[0] == "1":
				distance = distance
			else:
				distance = round(distance / int(pattern_d_ratio.findall(d_ratio)[0]))
			distance_values.append((distance,multiplier))
		distances_computed.append((id_, distance_values))
	return distances_computed

def sort_distances_computed(distances_computed):
	def sorter(x):
		sorting_list = list()
		for v,m in x[1]:
			sorting_list.append(v*m)
		return sorting_list
	return sorted(distances_computed, key=sorter)



def extract_n_neighbours_data(extracting_indices_1d, train_data_2d,k):
	if k > len(extracting_indices_1d):
		raise Exception('K Error; k > avaailable train data')
	if k == -1:
		return train_data_2d[extracting_indices_1d]
	else:
		return train_data_2d[extracting_indices_1d[:k]]

def extract_recommended_distances(sorted_distances_computed,k):
	if k > len(sorted_distances_computed):
		raise Exception('K Error; k > avaailable train data')
	if k == -1:
		return list(j for *_,j in sorted_distances_computed)
	else:
		return list(j for *_,j in sorted_distances_computed[:k])

def extract_recommeded_indices(sorted_distances_computed, k):

	if k > len(sorted_distances_computed):
		raise Exception('K Error; k > available train data')
	if k == -1:
		return list(i for i,*_ in sorted_distances_computed)
	else:
		return list(i for i,*_ in sorted_distances_computed[:k])


def compile_final_output(label_data, recommended_data_2d,recommeded_distances,recommeded_names):
	final_label = [*label_data,'Movie Title']
	final_data = np.hstack((recommended_data_2d,recommeded_names[:,np.newaxis]))
	return pd.DataFrame(data=final_data, columns=final_label)


def model_knn(k_neighbours, train_data_2d, query_data_1d,train_data_movie_titles_1d,distance_template, label_data=None):
	distances_computed = compute_distances(query_data_1d,train_data_2d,distance_template)
	sorted_distances_computed = sort_distances_computed(distances_computed)
	extracting_indices = list(i for i,*_ in sorted_distances_computed)
	recommended_data_2d = extract_n_neighbours_data(extracting_indices, train_data_2d, k_neighbours)
	recommeded_distances = extract_recommended_distances(sorted_distances_computed, k_neighbours)
	recommeded_indices = extract_recommeded_indices(sorted_distances_computed,k_neighbours)
	recommeded_names = train_data_movie_titles_1d[recommeded_indices]
	return compile_final_output(label_data,recommended_data_2d,recommeded_distances,recommeded_names),recommeded_distances



<h3>Reading The Movies Data</h3>

In [7]:
hollywood_data = pd.read_csv(r"data\hollywood.csv",error_bad_lines=False,index_col=False,warn_bad_lines=False)
bollywood_data = pd.read_csv(r"data\bollywood.csv",error_bad_lines=False,index_col=False,warn_bad_lines=False)
british_data = pd.read_csv(r"data\british.csv",error_bad_lines=False,index_col=False,warn_bad_lines=False)
chinese_data = pd.read_csv(r"data\chinese.csv",error_bad_lines=False,index_col=False,warn_bad_lines=False)
korean_data = pd.read_csv(r"data\korean.csv",error_bad_lines=False,index_col=False,warn_bad_lines=False)
nollywood_data = pd.read_csv(r"data\nollywood.csv",error_bad_lines=False,index_col=False,warn_bad_lines=False)
thai_data = pd.read_csv(r"data\thai.csv",error_bad_lines=False,index_col=False,warn_bad_lines=False)
am_shows_data = pd.read_csv(r"data\america_tv_show.csv",error_bad_lines=False,index_col=False,warn_bad_lines=False)
k_drama_data = pd.read_csv(r"data\korean_drama.csv",error_bad_lines=False,index_col=False,warn_bad_lines=False)

<h3>Data Cleaning and Preprocessing</h3>

<h4>Defining parsing and filtering functions for year of production of the movies</h4>

In [5]:
#This function filters out movies that are produced before 2005(TV-SHOW Only)
def parse_year_show(x):
    try:
        return int(x.split('–')[0]) >= 2005
    except:
        return False
    
#This function filters out movies that are produced before a specified year(defaults to 1990)(MOVIES Only)
def filter_year(year, limit=1990):
    try:
        return int(year) >= limit
    except:
        try:
            return int(year.split('–')[0]) >= limit
        except:
            return False

#This function removes irregularities in movies year of production
def parse_year(year):
    try:
        return int(year)
    except:
        return int(year.split('–')[0])

<h4>Dropping duplicates from the movies data</h4>

In [6]:
#droping duplicates
hollywood_data.drop_duplicates(subset=['imdbID'], keep='first', inplace=True, ignore_index=True)
bollywood_data.drop_duplicates(subset=['imdbID'], keep='first', inplace=True, ignore_index=True)
nollywood_data.drop_duplicates(subset=['imdbID'], keep='first', inplace=True, ignore_index=True)
british_data.drop_duplicates(subset=['imdbID'], keep='first', inplace=True, ignore_index=True)
chinese_data.drop_duplicates(subset=['imdbID'], keep='first', inplace=True, ignore_index=True)
korean_data.drop_duplicates(subset=['imdbID'], keep='first', inplace=True, ignore_index=True)
thai_data.drop_duplicates(subset=['imdbID'], keep='first', inplace=True, ignore_index=True)
k_drama_data.drop_duplicates(subset=['imdbID'], keep='first', inplace=True, ignore_index=True)
am_shows_data.drop_duplicates(subset=['imdbID'], keep='first', inplace=True, ignore_index=True)


<h4>Collating the movies search parameters(title,year,actors,plot,imdbID) into data structures for easy access</h4>

In [7]:
#getting the part of the movies data needed for querying 
dict_tags_movies = {"hollywood":(hollywood_data['title'].astype(np.str_).str.lower().str.replace(".", "").str.replace(",", "").str.replace("-", "") +", " + hollywood_data['year'].astype(np.str_).str.lower().str.replace(".", "").str.replace(",", "").str.replace("-", "") + ", " + hollywood_data['actors'].astype(np.str_).str.lower().str.replace(".", "").str.replace(",", "").str.replace("-", "")+ ", " + hollywood_data['Plot'].astype(np.str_).str.lower().str.replace(".", "").str.replace(",", "").str.replace("-", "") + ":" + hollywood_data['imdbID'].astype(np.str_)).values,
                         "bollywood": (bollywood_data['title'].astype(np.str_).str.lower().str.replace(".", "").str.replace(",", "").str.replace("-", "") +", " + bollywood_data['year'].astype(np.str_).str.lower().str.replace(".", "").str.replace(",", "").str.replace("-", "") +", " + bollywood_data['actors'].astype(np.str_).str.lower().str.replace(".", "").str.replace(",", "").str.replace("-", "")+", " + bollywood_data['Plot'].astype(np.str_).str.lower().str.replace(".", "").str.replace(",", "").str.replace("-", "")+":" + bollywood_data['imdbID'].astype(np.str_)).values,
                         "nollywood": (nollywood_data['title'].astype(np.str_).str.lower().str.replace(".", "").str.replace(",", "").str.replace("-", "") +", " + nollywood_data['year'].astype(np.str_).str.lower().str.replace(".", "").str.replace(",", "").str.replace("-", "") +", " + nollywood_data['actors'].astype(np.str_).str.lower().str.replace(".", "").str.replace(",", "").str.replace("-", "")+", " + nollywood_data['Plot'].astype(np.str_).str.lower().str.replace(".", "").str.replace(",", "").str.replace("-", "")+":" + nollywood_data['imdbID'].astype(np.str_)).values,
                         "british movies": (british_data['title'].astype(np.str_).str.lower().str.replace(".", "").str.replace(",", "").str.replace("-", "") +", " + british_data['year'].astype(np.str_).str.lower().str.replace(".", "").str.replace(",", "").str.replace("-", "") +", " + british_data['actors'].astype(np.str_).str.lower().str.replace(".", "").str.replace(",", "").str.replace("-", "")+", " + british_data['Plot'].astype(np.str_).str.lower().str.replace(".", "").str.replace(",", "").str.replace("-", "")+":" + british_data['imdbID'].astype(np.str_)).values,
                         "chinese movies" :(chinese_data['title'].astype(np.str_).str.lower().str.replace(".", "").str.replace(",", "").str.replace("-", "") +", " + chinese_data['year'].astype(np.str_).str.lower().str.replace(".", "").str.replace(",", "").str.replace("-", "") +", " + chinese_data['actors'].astype(np.str_).str.lower().str.replace(".", "").str.replace(",", "").str.replace("-", "")+", " + chinese_data['Plot'].astype(np.str_).str.lower().str.replace(".", "").str.replace(",", "").str.replace("-", "")+":" + chinese_data['imdbID'].astype(np.str_)).values,
                         "korean movies": (korean_data['title'].astype(np.str_).str.lower().str.replace(".", "").str.replace(",", "").str.replace("-", "") +", " + korean_data['year'].astype(np.str_).str.lower().str.replace(".", "").str.replace(",", "").str.replace("-", "") +", " + korean_data['actors'].astype(np.str_).str.lower().str.replace(".", "").str.replace(",", "").str.replace("-", "")+", " + korean_data['Plot'].astype(np.str_).str.lower().str.replace(".", "").str.replace(",", "").str.replace("-", "")+":" + korean_data['imdbID'].astype(np.str_)).values,
                         "thai movies" : (thai_data['title'].astype(np.str_).str.lower().str.replace(".", "").str.replace(",", "").str.replace("-", "") +", " + thai_data['year'].astype(np.str_).str.lower().str.replace(".", "").str.replace(",", "").str.replace("-", "") +", " + thai_data['actors'].astype(np.str_).str.lower().str.replace(".", "").str.replace(",", "").str.replace("-", "")+", " + thai_data['Plot'].astype(np.str_).str.lower().str.replace(".", "").str.replace(",", "").str.replace("-", "")+":" + thai_data['imdbID'].astype(np.str_)).values
                        }

dict_tags_series = {"tv-show": (am_shows_data['title'].astype(np.str_).str.lower().str.replace(".", "").str.replace(",", "").str.replace("-", "") +", " + am_shows_data['year'].astype(np.str_).str.lower().str.replace(".", "").str.replace(",", "").str.replace("-", "")+", " + am_shows_data['actors'].astype(np.str_).str.lower().str.replace(".", "").str.replace(",", "").str.replace("-", "")+", " + am_shows_data['Plot'].astype(np.str_).str.lower().str.replace(".", "").str.replace(",", "").str.replace("-", "")+ ":" + am_shows_data['imdbID'].astype(np.str_)).values,
                    "korean drama": (k_drama_data['title'].astype(np.str_).str.lower().str.replace(".", "").str.replace(",", "").str.replace("-", "") +", " + k_drama_data['year'].astype(np.str_).str.lower().str.replace(".", "").str.replace(",", "").str.replace("-", "")+", " + k_drama_data['actors'].astype(np.str_).str.lower().str.replace(".", "").str.replace(",", "").str.replace("-", "")+", " + k_drama_data['Plot'].astype(np.str_).str.lower().str.replace(".", "").str.replace(",", "").str.replace("-", "")+ ":" + k_drama_data['imdbID'].astype(np.str_)).values
                    }    

<h4>Filtering and Parsing the movies data by year of release</h4>

In [8]:
#removing hollywood movies produced before 1990
hollywood_data = hollywood_data[hollywood_data["year"]>=1990]

#removing bollywood movies produced before 1990
bollywood_data = bollywood_data[bollywood_data["year"]>=1990]

#removing british movies produced before 1990
british_data = british_data[british_data["year"].apply(lambda x: filter_year(x))]

#removing chinese movies produced before 1970
chinese_data = chinese_data[chinese_data["year"].apply(lambda x: filter_year(x,1970))]

#removing nollywood movies produced before 1990
nollywood_data = nollywood_data[nollywood_data["year"].apply(lambda x: filter_year(x))]

#removing korean movies produced before 1990
korean_data = korean_data[korean_data["year"].apply(lambda x: filter_year(x))]

#removing thai movies produced before 1990
thai_data = thai_data[thai_data["year"].apply(lambda x: filter_year(x))]

#removing hollywood and bollywood movies produced before 1990 AND Filtering invalid years out
bollywood_data = bollywood_data[bollywood_data["year"].apply(lambda x: filter_year(x))]
hollywood_data = hollywood_data[hollywood_data["year"].apply(lambda x: filter_year(x))]

#Parsing the movies data and fixing the irregularities in them
british_data['year'] = british_data["year"].apply(parse_year).astype(np.int64)
chinese_data['year'] = chinese_data["year"].apply(parse_year).astype(np.int64)
nollywood_data['year'] = nollywood_data["year"].apply(parse_year).astype(np.int64)
korean_data['year'] = korean_data["year"].apply(parse_year).astype(np.int64)
thai_data['year'] = thai_data["year"].apply(parse_year).astype(np.int64)
bollywood_data['year'] = bollywood_data["year"].apply(parse_year).astype(np.int64)
hollywood_data['year'] = hollywood_data["year"].apply(parse_year).astype(np.int64)

<h4>Collating the movies data into data structures for easy access</h4>

In [9]:
dict_data = {
	"hollywood":hollywood_data,
	"bollywood":bollywood_data,
	"tv-show": am_shows_data,
    "british movies":british_data,
    "chinese movies":chinese_data,
    "korean movies":korean_data,
    "nollywood":nollywood_data,
    "thai movies":thai_data,
    "korean drama":k_drama_data
}

<h3>Getting input from user</h3>

In [10]:
movie_tag = input('Movie name or tag e.g (john wick, vampire diaries, thanos, vibranium wakanda);\n')
movie_type = input(f'Movie Type; ({", ".join(dict_data.keys())})\n')
print(f"\n\n{movie_tag}\n{movie_type}")
if movie_type.lower() in ["tv-show","korean drama"]:
    movie_type_ = "series"
else:
    movie_type_ = 'movie'

Movie name or tag e.g (john wick, vampire diaries, thanos, vibranium wakanda);
Rio
Movie Type; (hollywood, bollywood, tv-show, british movies, chinese movies, korean movies, nollywood, thai movies, korean drama)
hollywood


Rio
hollywood


<h3>Performing the movie query</h3>

In [11]:
standard_query_data = get_name_movie(movie_tag,dict_tags_movies[movie_type.lower()] if movie_type_ == "movie" else dict_tags_series[movie_type.lower()] ,movie_type_)
if standard_query_data[0]:
    print('Query Successful', standard_query_data, sep="\n")
    queried_movie_type = movie_type.lower()
    standard_query_data = standard_query_data[0]
else:
    print('Query Error', standard_query_data, sep="\n")
    queried_movie_type = None
    standard_query_data = None

Query Successful
(['tt1436562', 'Rio', 'http://www.imdb.com/title/tt1436562/', 6.9, 72.0, 63.0, 208829, 5760, '2011', 1, 3, 30, 0, 0, 1, 1, 0, 1, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 'Carlos Saldanha', 'Carlos Saldanha (story)- Earl Richey Jones (story)- Todd R. Jones (story)- Don Rhymer (screenplay)- Joshua Sternin (screenplay)- Jennifer Ventimilia (screenplay)- Sam Harper (screenplay)', 'Karen Disher- Jason Fricchione- Sofia Scarpa Saldanha- Leslie Mann', 'Blue Sky Studios', 'USA- Brazil', 'Arabic- English- Portuguese', 'movie', 'When Blu- a domesticated macaw from small-town Minnesota- meets the fiercely independent Jewel- he takes off on an adventure to Rio de Janeiro with the bird of his dreams.', '$143-619-809', 'G', 'Animation- Adventure- Comedy- Crime- Family- Musical'], 'API Search', None)


<h3>Data Processing for machine learning algorithm application</h3>

<h4>Defining Processing Function</h4>

In [12]:
#This function extracts the value of a special nomination from the given string x by using regex
def extract_nom_val(x,nom):
    try:
        res = re.findall(fr'(\d+?):{nom}', x, re.IGNORECASE)
        if res:
            return int(res[0]) 
        else:
            return 0
    except:
        pass

<h4>Data Processing line of codes </h4>

In [13]:
#Getting the needed movie data from the compiled movies data
standard_train_data = dict_data[queried_movie_type.lower()]
#Getting train_data, a copied version of standard_train_data
train_data = standard_train_data.copy()

#SUB-categorizing the queried movie type into series or movie
if queried_movie_type.lower() in ["tv-show","korean drama"]:
    queried_movie_type = "series"
else:
    queried_movie_type = 'movie'

#Converting the year of release value for the queried movie data into integer(MOVIES Only)
if queried_movie_type == 'movie':
    try:
        standard_query_data[8] = int(standard_query_data[8].split('–')[0])
    except:
        pass
    
#Removing the queried movie data from the train data(IF EXIST)
train_data = train_data[train_data['imdbID'] != standard_query_data[0]]

#Appending the queried movie data to the train_data for combined data processing, this forms a new data called train_data_with_query
train_data_with_query = train_data.append(pd.Series(index=train_data.columns, data=standard_query_data),ignore_index=True)

#Dropping the box office attribute from train_data_with_query(MOVIES only)
if queried_movie_type != "series":
    train_data_with_query.drop('BoxOffice',axis=1,inplace=True)

    
#Treating Missing Values in train_data_with_query

#-------Columns or Attributes that will have their missing values replaced with Unknown
column_unknown = ['directors','writers', 'actors', 'production', 'country', 'language','Plot','Rated','Type','Genre']
for c in column_unknown:
    try:
        train_data_with_query[c].fillna('Unknown',inplace=True)
    except:
        pass
    
#-------Columns or Attributes that will have their missing values replaced with their respective medians
if queried_movie_type == 'movie':
    column_median = ['imdbRating', 'rottenTomatoRating','metacriticRating','duration','year']
else:
    column_median = ['imdbRating','duration', 'totalSeasons']
for c in column_median:
    try:
        train_data_with_query[c].fillna(train_data_with_query[c].median(),inplace=True)
    except Exception as err:
        raise err
        
#-------Columns or Attributes that will have their missing values replaced with zero(0)        
if queried_movie_type == 'movie':
    column_zero = ['oscarNominations',
           'noOfAwards', 'noOfNominations','imdbVotes']
else:
    column_zero = ['noOfNominations','imdbVotes','noOfAwards']
for c in column_zero:
    try:
        train_data_with_query[c].fillna(0,inplace=True)
    except:
        pass
    
#-------Columns or Attributes that will have their missing values replaced with their respective mode
if queried_movie_type == 'movie':
    column_mode = []
else:
    column_mode = ['year']
for c in column_mode:
    try:
        train_data_with_query[c].fillna(train_data_with_query[c].mode()[0],inplace=True)
    except Exception as err:
        raise err

        
#Extracting the Special Award counts from the movies data(SERIES ONLY)
if queried_movie_type.lower() == 'series':
    list_special_awards = ['Golden Globe', 'Primetime Emmy']
    for spec_nom in list_special_awards:
        train_data_with_query[spec_nom] = train_data_with_query['specialNominations'].apply(lambda x: extract_nom_val(x,spec_nom))
        column_zero.append(spec_nom)
        train_data_with_query[spec_nom].fillna(0,inplace=True)

#Parsing the year column of the movies data(SERIES ONLY)
if queried_movie_type == 'series':
    train_data_with_query['year'] = train_data_with_query['year'].apply(lambda x: int(x.split('–')[0]))
    column_zero.append('year');

<h4>Compiling labels and id of the columns in train_data_with_query</h4>

In [14]:
dict_cols_init = dict((j,i) for i,j in enumerate(train_data_with_query.columns))

<h4>Converting the imdbRating attribute of the train_data_with_query into float data type </h4>

In [15]:
train_data_with_query.iloc[-1,3] = np.float64(train_data_with_query.iloc[-1,3])
train_data_with_query['imdbRating']*=10

<h4>getting the queried movie data back from the train_data_with_query</h4>

In [16]:
standard_query_data = train_data_with_query.iloc[-1,:]

<h4>Defining a stop words removal algorithm</h4>

In [17]:
stop_words = {'ourselves', 'hers', 'between', 'yourself', ',', '.', ':', 'but', 'again', 'there', 'about', 'once', 'during', 'out', 'very', 'having', 'with', 'they', 'own', 'an', 'be', 'some', 'for', 'do', 'its', 'yours', 'such', 'into', 'of', 'most', 'itself', 'other', 'off', 'is', 's', 'am', 'or', 'who', 'as', 'from', 'him', 'each', 'the', 'themselves', 'until', 'below', 'are', 'we', 'these', 'your', 'his', 'through', 'don', 'nor', 'me', 'were', 'her', 'more', 'himself', 'this', 'down', 'should', 'our', 'their', 'while', 'above', 'both', 'up', 'to', 'ours', 'had', 'she', 'all', 'no', 'when', 'at', 'any', 'before', 'them', 'same', 'and', 'been', 'have', 'in', 'will', 'on', 'does', 'yourselves', 'then', 'that', 'because', 'what', 'over', 'why', 'so', 'can', 'did', 'not', 'now', 'under', 'he', 'you', 'herself', 'has', 'just', 'where', 'too', 'only', 'myself', 'which', 'those', 'i', 'after', 'few', 'whom', 't', 'being', 'if', 'theirs', 'my', 'against', 'a', 'by', 'doing', 'it', 'how', 'further', 'was', 'here', 'than'}
def remove_stop_words(x):
    return not x in stop_words

<h4>Getting some data from the queried movie data, these colleceted data are later used for combined processing of the whole train data</h4>

In [18]:
actors_set = list(i.strip() for i in standard_query_data[dict_cols_init['actors']].split('-'))
language_set = list(i.strip() for i in standard_query_data[dict_cols_init['language']].split('-'))
genre_set = list(i.strip() for i in standard_query_data[dict_cols_init['Genre']].split('-'))
country_set = list(i.strip() for i in standard_query_data[dict_cols_init['country']].split('-'))
plot_set = list(filter(remove_stop_words,set(re.sub(r'(^\W|\W$)', '', i).lower() for i in standard_query_data[dict_cols_init['Plot']].split())))
title_set = list(filter(remove_stop_words,set(re.sub(r'(^\W|\W$)', '', i).lower() for i in standard_query_data[dict_cols_init['title']].split())))

<h4>Defining some processing functions </h4>
<h4>These functions returns the length of the intersection of two sets of data</h4>

In [19]:
def get_closeness_list(x, to_compare):
    a = (len(set(to_compare).intersection(set(x)))/len(to_compare))*(100-len(to_compare))
    b = 0
    for id_, i in enumerate(x):
        try:
            if to_compare[id_] == i:
                b += 1
            else:
                break
        except:
            break
    return a + b

def get_plot_match(x):
    plot_list = list(filter(remove_stop_words,set(re.sub(r'(^\W|\W$)', '', i).lower() for i in x.split())))
    a = (len(set(plot_set).intersection(set(plot_list)))/len(plot_set))*100
    return a

def get_title_match(x):
    title_list = list(filter(remove_stop_words,set(re.sub(r'(^\W|\W$)', '', i).lower() for i in x.split())))
    a = (len(set(title_set).intersection(set(title_list)))/len(title_set))*100
    return a

<h4>Performing similarities matching between some attributes of the standard_query_data and the train_data_with_query</h4>

In [20]:
if queried_movie_type == 'series':
    train_data_with_query['actor_fuzz'] = train_data_with_query['actors'].apply(lambda x:fuzz.token_sort_ratio(standard_query_data[dict_cols_init['actors']],x))
else:
    train_data_with_query['actor_fuzz'] = train_data_with_query['actors'].apply(lambda x: get_closeness_list(list(i.strip() for i in x.split('-')), actors_set))
dict_cols_init['actor_fuzz'] = len(dict_cols_init.values())
column_median.append('actor_fuzz')
train_data_with_query['language_fuzz'] = train_data_with_query['language'].apply(lambda x:get_closeness_list(list(i.strip() for i in x.split('-')), language_set))
dict_cols_init['language_fuzz'] = len(dict_cols_init.values())
column_median.append('language_fuzz')
train_data_with_query['rated_fuzz'] = train_data_with_query['Rated'].apply(lambda x:fuzz.token_sort_ratio(standard_query_data[dict_cols_init['Rated']],x))
dict_cols_init['rated_fuzz'] = len(dict_cols_init.values())
column_median.append('rated_fuzz')
train_data_with_query['type_fuzz'] = train_data_with_query['Type'].apply(lambda x:fuzz.token_sort_ratio(standard_query_data[dict_cols_init['Type']],x))
dict_cols_init['type_fuzz'] = len(dict_cols_init.values())
column_median.append('type_fuzz')
if queried_movie_type == 'series':
    train_data_with_query['plot_fuzz'] = train_data_with_query['Plot'].apply(lambda x:fuzz.token_sort_ratio(standard_query_data[dict_cols_init['Plot']],x))
else:
    train_data_with_query['plot_fuzz'] = train_data_with_query['Plot'].apply(get_plot_match)

dict_cols_init['plot_fuzz'] = len(dict_cols_init.values())
column_median.append('plot_fuzz')
if queried_movie_type == 'series':
    train_data_with_query['genre_fuzz'] = train_data_with_query['Genre'].apply(lambda x:fuzz.token_sort_ratio(standard_query_data[dict_cols_init['Genre']],x))
else:
    train_data_with_query['genre_fuzz'] = train_data_with_query['Genre'].apply(lambda x: get_closeness_list(list(i.strip() for i in x.split('-')), genre_set))
dict_cols_init['genre_fuzz'] = len(dict_cols_init.values())
column_median.append('genre_fuzz')
# train_data_with_query['country_fuzz'] = train_data_with_query['country'].apply(lambda x:fuzz.token_sort_ratio(standard_query_data[dict_cols_init['country']],x))
train_data_with_query['country_fuzz'] = train_data_with_query['country'].apply(lambda x: get_closeness_list(list(i.strip() for i in x.split('-')), country_set))
dict_cols_init['country_fuzz'] = len(dict_cols_init.values())
column_median.append('country_fuzz')
# train_data_with_query['title_fuzz'] = train_data_with_query['title'].apply(lambda x:fuzz.token_sort_ratio(standard_query_data[dict_cols_init['title']],x))
train_data_with_query['title_fuzz'] = train_data_with_query['title'].apply(get_title_match)
dict_cols_init['title_fuzz'] = len(dict_cols_init.values())
column_median.append('title_fuzz')

def scale_data(x, col):
    return ((x/train_data_with_query[col].max()) * 100)

if queried_movie_type == 'series':
    train_data_with_query['plot_fuzz'] = train_data_with_query['plot_fuzz'].apply(lambda x: 0 if (x <70) else x)
    train_data_with_query['genre_fuzz'] = train_data_with_query['genre_fuzz'].apply(lambda x: 0 if (x <70) else x)
    train_data_with_query['rated_fuzz'] = train_data_with_query['rated_fuzz'].apply(lambda x: 0 if (x <100) else x)
    train_data_with_query['actor_fuzz'] = train_data_with_query['actor_fuzz'].apply(lambda x: 0 if (x <80) else x)
    train_data_with_query['title_fuzz'] = train_data_with_query['title_fuzz'].apply(lambda x: 0 if (x <80) else x)
    train_data_with_query['noOfAwards'] =  train_data_with_query['noOfAwards'].apply(lambda x: scale_data(x, 'noOfAwards'))
    train_data_with_query['noOfNominations'] =  train_data_with_query['noOfAwards'].apply(lambda x: scale_data(x, 'noOfNominations'))
    train_data_with_query['imdbVotes'] =  train_data_with_query['imdbVotes'].apply(lambda x: scale_data(x, 'imdbVotes'))
    # train_data_with_query['Primetime Emmy'] =  train_data_with_query['Primetime Emmy'].apply(lambda x: scale_data(x, 'Primetime Emmy'))
    # train_data_with_query['Golden Globe'] =  train_data_with_query['Primetime Emmy'].apply(lambda x: scale_data(x, 'Golden Globe'))
else:
    train_data_with_query['title_fuzz'] = train_data_with_query['title_fuzz'].apply(lambda x: 0 if (x <80) else x)

<h4>Filtering and Parsing the Genres attributes of train_data_with_query</h4>

In [21]:
column_genres = ['Action',
       'Adult', 'Adventure', 'Animation', 'Biography', 'Comedy', 'Crime',
       'Documentary', 'Drama', 'Family', 'Fantasy', 'FilmNoir', 'GameShow',
       'History', 'Horror', 'Music', 'Musical', 'Mystery', 'News', 'RealityTV',
       'Romance', 'SciFi', 'Short', 'Sport', 'TalkShow', 'Thriller', 'War',
       'Western']

def parse_genre(x):
    try:
        if int(x) in [0,1]:
            return int[x]
        else:
            return 0;
    except:
        return 0
for col in column_genres:
    train_data_with_query[col] = train_data_with_query[col].fillna(0)

<h4>Feature Scaling</h4>

<h4>MinMax Scaler is used for movies while Standard Scaler is used for series</h4>

In [22]:
if queried_movie_type == 'movie':
    scaler = MinMaxScaler()
else:
    scaler = StandardScaler()
to_be_standardized_features = column_median+column_zero
to_be_standardized_data = train_data_with_query[to_be_standardized_features]

train_data_with_query_standardized = scaler.fit_transform(to_be_standardized_data)
for id_, c in enumerate(to_be_standardized_features):
    train_data_with_query[c] = train_data_with_query_standardized[:,id_]

<h4>Compiling labels and id of the columns in train_data_with_query</h4>

In [23]:
dict_cols_final = dict((j,i) for i,j in enumerate(train_data_with_query.columns))

<h4>Creating an algorithm template</h4>
<h4>This template specifies the attributes of the movies data on which the algorithm will be applied</h4>
<h4>This attributes are specified with their respective index no</h4>

In [24]:
if queried_movie_type == 'movie':
    distance_template = [
        [1, [50,52,53,54,55,3,4,5,6,7,8,10,11,51], 'e','d1',1]
    ]
else:
    distance_template = [
        [1, [4,54,55,56], 'e', 'd1', 1]
    ]

<h4>Getting the data needed to be passed to the fitting function</h4>

In [25]:
train_data_2d = train_data_with_query.iloc[:-1,:].values
labels = train_data_with_query.columns
train_data_movie_titles_1d = train_data_with_query.iloc[:-1,:]['title'].values
query_data_1d = train_data_with_query.iloc[-1,:].values

<h3>Application of Algorithm(Calling the fitting function)</h3>

In [26]:
reply = model_knn(-1, *[train_data_2d, query_data_1d,train_data_movie_titles_1d,distance_template, labels])

<h4>Displaying the fitting results</h4>

In [27]:
print(f'Queried Movie; {movie_tag}')
print(f'Closest Match; {query_data_1d[1]}, {standard_query_data[8]}')
print(f'Recommended Movies;;;')
to_extract = standard_train_data.set_index('imdbID').loc[reply[0]['imdbID'],:]
pd.concat([reply[0]['imdbID'].apply(lambda x: f'{to_extract.loc[x,"title"]}, {to_extract.loc[x,"year"]}, {x}'), pd.Series(reply[1], name='Similarity Score')], axis=1).head(50)

Queried Movie; Rio
Closest Match; Rio, 2011
Recommended Movies;;;


,imdbID,Similarity Score
0,"Rio 2, 2014, tt2357291","[(1.2829200413861803, 1)]"
1,"Ice Age: Dawn of the Dinosaurs, 2009, tt1080016","[(1.5348904294720647, 1)]"
2,"The Princess and the Frog, 2009, tt0780521","[(1.5512377102535446, 1)]"
3,"Madagascar, 2005, tt0351283","[(1.5595815072365575, 1)]"
4,"Winnie the Pooh, 2011, tt1449283","[(1.5604986698956342, 1)]"
5,"The Polar Express, 2004, tt0338348","[(1.5654753808043533, 1)]"
6,"Meet the Robinsons, 2007, tt0396555","[(1.5750173181502474, 1)]"
7,"The Book of Life, 2014, tt2262227","[(1.5842962700584073, 1)]"
8,"Scooby-Doo! Music of the Vampire, 2012, tt2162709","[(1.590023677889578, 1)]"
9,"Toy Story 2, 1999, tt0120363","[(1.6033971713758477, 1)]"
